In [600]:
#python example to infer document vectors from trained doc2vec model
import gensim.models as g
import codecs
from scipy import spatial
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import preprocessing


In [810]:
def create_dataframe(contest,dvec_model,g_truth):

    df = OrderedDict()
    file_class =[]
    csv = pd.read_csv('../Data/'+str(contest)+'/standings_statistics.csv',sep='\t')#,skiprows=1,header=None,names=['ParticipantID','Rank','Country','ProblemA_id','ProblemA_memory','ProblemA_time','ProblemA_language','ProblemB_id','ProblemB_memory','ProblemB_time','ProblemB_language','ProblemC_id','ProblemC_memory','ProblemC_time','ProblemC_language','ProblemD_id','ProblemD_memory','ProblemD_time','ProblemD_language','ProblemE_id','ProblemE_memory','ProblemE_time','ProblemE_language','ProblemF_id','ProblemF_memory','ProblemF_time','ProblemF_language','ProblemG_id','ProblemG_memory','ProblemG_time','ProblemG_language'])
    if g_truth=='country':
        China_participants = np.array(csv.loc[(csv['Country']=='China'),['ParticipantID']])
        China_participants = [str(i[0]) for i in China_participants]
        RoW_participants = np.array(csv.loc[(csv['Country']!='China') & (csv['Country']!='-'),['ParticipantID']])
        RoW_participants =  [str(i[0]) for i in RoW_participants]
        p_flag=[]
    for i in os.listdir('../Data/'+str(contest)+'/all_xml_simplified/'):
        f_xml_string = open('../Data/'+str(contest)+'/all_xml_simplified/'+str(i),'r').read()
        f_xml_list = f_xml_string.strip().split()    
        file_id = str(i).strip('.xml').split("_")[1]
        participant_id = str(i).strip('.xml').split("_")[0]
        file_id = str(file_id)
        participant_id = str(participant_id)
        if g_truth=='problem':
            df[i] = dvec_model.infer_vector(f_xml_list)
            if file_id in csv['Problem1_id'].to_list():
                file_class.append(0)
            elif file_id in csv['Problem2_id'].to_list():
                file_class.append(1)
            elif file_id in csv['Problem3_id'].to_list():
                file_class.append(2)
            elif file_id in csv['Problem4_id'].to_list():
                file_class.append(3)
            elif file_id in csv['Problem5_id'].to_list():
                file_class.append(4)
            elif file_id in csv['Problem6_id'].to_list():
                file_class.append(5)
            elif file_id in csv['Problem7_id'].to_list():
                file_class.append(6)
        elif g_truth=='country':  #making vectors for participants who have given their country
            if participant_id in China_participants and participant_id not in p_flag:
                file_class.append(0)
                p_flag.append(participant_id)
                df[i] = dvec_model.infer_vector(f_xml_list)    
            elif participant_id in RoW_participants and participant_id not in p_flag:
                file_class.append(1)
                p_flag.append(participant_id)
                df[i] = dvec_model.infer_vector(f_xml_list)
        elif g_truth=='rating':

            if np.array(csv.loc[csv['ParticipantID'].astype(str)==participant_id,['Color']]=='blue')[0][0]:
                file_class.append(0)
                df[i] = dvec_model.infer_vector(f_xml_list)
            elif np.array(csv.loc[csv['ParticipantID'].astype(str)==participant_id,['Color']]=='violet')[0][0]:
                file_class.append(1)
                df[i] = dvec_model.infer_vector(f_xml_list)
            else:
                file_class.append(3)
                df[i] = dvec_model.infer_vector(f_xml_list)
                
    df = pd.DataFrame(df)
    print(len(file_class))
    df = df.assign(rating=file_class) 
            
            
                
    return df


    





def cluster_vecs(n_clusters,data):
    kmeans_model = KMeans(init='k-means++', n_clusters=n_clusters, n_init=n_clusters,n_jobs=-1)
    mdl = kmeans_model.fit(data)
    return mdl

def intersection(a,b):
    return len(list(set(a) & set(b)))

def train_test_split(data,labels,n_class,tr_cut):
    data_train = data[:round(tr_cut*(data.shape[0]))]
    labels_train = labels[:round(tr_cut*(labels.shape[0]))]
    data_test = data[round(tr_cut*(data.shape[0])):]
    labels_test = labels[round(tr_cut*(labels.shape[0])):]
    class_weight=compute_class_weight(None, np.arange(0,n_class), labels)
    cw_dict = {}
    for i in range(len(class_weight)):
        cw_dict[i] = class_weight[i]
    return data_train,labels_train,data_test,labels_test,cw_dict
        
def log_reg(data_train,labels_train,data_test,labels_test,n_class,cw_dict):
    if n_class>2:
        log_reg = LogisticRegression(multi_class='multinomial',solver='lbfgs',n_jobs=-1,class_weight=cw_dict)
    else:
        log_reg = LogisticRegression(class_weight=cw_dict,n_jobs=-1,solver='lbfgs')
    log_reg.fit(data_train,labels_train)
    print("Training accuracy-")
    print(accuracy_score(labels_train,log_reg.predict(data_train)))
    print("Testing accuracy-")
    print(accuracy_score(labels_test,log_reg.predict(data_test)))
    print("Confusion matrix -")
    print(confusion_matrix(labels_test,log_reg.predict(data_test)))
    return log_reg

def rw(data_train,labels_train,data_test,labels_test,class_weight):
    rand_f_mdl = RandomForestClassifier(class_weight=class_weight,n_estimators=70,n_jobs=-1)
    rand_f_mdl.fit(data_train,labels_train)
    print("Training accuracy-")
    print(accuracy_score(labels_train,rand_f_mdl.predict(data_train)))
    print("Testing accuracy-")
    print(accuracy_score(labels_test,rand_f_mdl.predict(data_test)))
    print("Confusion matrix -")
    print(confusion_matrix(labels_test,rand_f_mdl.predict(data_test)))
    return rand_f_mdl
    

In [62]:
#parameters
model="model.bin"
#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000
#load model
m = g.Doc2Vec.load(model)


In [189]:
contest = "Contest1244"

### Making vector representation of codes : doc2vec approach

#### Ground truth = Problem

In [745]:
#ground truth - problem
contest="Contest1244"
x,labels = create_dataframe(contest,m,'problem')

In [746]:
x.head()

,28931484_62469343.xml,28892317_62464524.xml,28935580_62479865.xml,28896263_62466856.xml,28907902_62492965.xml,28915172_62495881.xml,28925582_62483607.xml,28925802_62472104.xml,28928442_62471841.xml,28896263_62468691.xml,...,28934038_62466270.xml,28931145_62469113.xml,28895489_62494270.xml,28892632_62494762.xml,28892246_62488852.xml,28929551_62469835.xml,28921144_62490037.xml,28901345_62473166.xml,28906937_62469857.xml,28935245_62486535.xml
0,0.071939,0.137051,0.205771,0.017945,-0.140744,0.016290,0.180889,-0.018912,-0.229745,0.208379,...,0.176846,-0.131410,-0.466424,-0.335859,-0.386642,0.017522,-0.004480,-0.159693,-0.042362,-0.020040
1,0.235403,-0.387346,-0.166443,0.006666,-0.109202,0.394354,-0.106082,0.188718,0.057190,-0.149534,...,-0.314720,0.086998,0.027869,0.303845,-0.275131,-0.112838,0.220417,-0.123508,-0.005702,0.041996
2,0.376634,0.357073,0.464091,0.365565,0.110494,0.106424,0.590050,0.016449,0.262917,0.599230,...,0.278489,0.332038,0.274685,0.235892,0.114710,0.319632,0.331492,0.062465,0.527692,0.398717
3,-0.076796,-0.379196,-0.312362,-0.302054,0.402075,0.054991,-0.320338,0.047767,-0.143535,-0.217058,...,-0.396029,-0.081005,0.092153,0.020496,-0.122694,-0.233214,0.191856,-0.244204,0.117003,0.692643
4,-0.255217,0.523905,-0.131335,0.183350,-0.379611,-0.200223,-0.074517,-0.217734,-0.284450,0.249883,...,0.360158,-0.151306,-0.591685,-0.562919,-0.354222,0.031025,0.014136,-0.119674,-0.379185,-0.476034


In [747]:
data = np.array(x)
data = data.T


In [749]:
data.shape

(3717, 100)

## K-means on Doc2Vecs of codes

In [66]:
km_model = cluster_vecs(7,data)
pred_labels = km_model.labels_

In [67]:
pred_A = [i for i, x in enumerate(pred_labels) if x == 0]
pred_B = [i for i, x in enumerate(pred_labels) if x == 1]
pred_C = [i for i, x in enumerate(pred_labels) if x == 2]
pred_D = [i for i, x in enumerate(pred_labels) if x == 3]
pred_E = [i for i, x in enumerate(pred_labels) if x == 4]
pred_F = [i for i, x in enumerate(pred_labels) if x == 5]
pred_G = [i for i, x in enumerate(pred_labels) if x == 6]

true_A = [i for i, x in enumerate(labels) if x == 0]
true_B = [i for i, x in enumerate(labels) if x == 1]
true_C = [i for i, x in enumerate(labels) if x == 2]
true_D = [i for i, x in enumerate(labels) if x == 3]
true_E = [i for i, x in enumerate(labels) if x == 4]
true_F = [i for i, x in enumerate(labels) if x == 5]
true_G = [i for i, x in enumerate(labels) if x == 6]





In [68]:
conf={}
for i in range(len(pred_labels)):
    if pred_labels[i] not in conf.keys():
        conf[pred_labels[i]]=[0,0,0,0,0,0,0]
        
    else:
        conf[pred_labels[i]][labels[i]]+=1

In [69]:
for key,value in conf.items():
    conf[key][0] = conf[key][0]/len(true_A)
    conf[key][1] = conf[key][1]/len(true_B)
    conf[key][2] = conf[key][2]/len(true_C)
    conf[key][3] = conf[key][3]/len(true_D)
    conf[key][4] = conf[key][4]/len(true_E)
    conf[key][5] = conf[key][5]/len(true_F)
    conf[key][6] = conf[key][6]/len(true_G)
    
    

In [70]:
df_confusion = pd.DataFrame(conf)

In [71]:
df_confusion = df_confusion.reindex(sorted(df_confusion.columns), axis=1)

In [72]:
df_confusion

,0,1,2,3,4,5,6
0,0.119622,0.025184,0.495278,0.037775,0.000000,0.003148,0.317943
1,0.132214,0.597062,0.033578,0.121721,0.002099,0.009444,0.101784
2,0.150150,0.039039,0.453453,0.036036,0.006006,0.006006,0.303303
3,0.074911,0.017836,0.001189,0.147444,0.731272,0.023781,0.002378
4,0.135189,0.035785,0.019881,0.115308,0.009940,0.638171,0.043738
5,0.126582,0.075949,0.000000,0.151899,0.025316,0.607595,0.012658
6,0.145455,0.163636,0.018182,0.163636,0.181818,0.254545,0.072727


## Logistic Regression approach (Multi-class classification)

#### Ground Truth = Problem 

In [73]:
labels = np.array(labels)
labels.shape

(3717,)

In [74]:
log_reg(data,labels,7,0.5)  #args - data,labels,n_classes, 60% for training

Testing accuracy-
0.8886498117267349


#### Ground truth = Country

In [75]:
x_country,labels_country = create_dataframe("Contest1244",m,'country')

['28925463', '28926600', '28935245', '28933193', '28936301', '28931633', '28928771', '28910382', '28934789', '28935540', '28898322', '28910315', '28925315', '28928624', '28894110', '28931295', '28931516', '28935243', '28928881', '28924921', '28935444', '28928852', '28925569', '28933631', '28927274', '28931828', '28909082', '28909624', '28935979', '28932347', '28928872', '28930148', '28898167', '28909125', '28935135', '28933702', '28894428', '28936104', '28928236', '28927460', '28908054', '28898109', '28911518', '28936447', '28927259', '28897551', '28897165', '28929699', '28907902', '28929620', '28901623', '28934897', '28904365', '28929497', '28898321', '28910995', '28924799', '28928682', '28912022', '28932160', '28929588', '28936135', '28935782', '28931220', '28933664', '28900318', '28934270', '28914024', '28928765', '28932796', '28929516', '28897193', '28926538', '28931554', '28927819', '28925566', '28900970', '28927964', '28933797', '28906844', '28933250', '28934416', '28932086', '28

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [76]:
data_country = np.array(x_country)
data_country = data_country.T
labels_country = np.array(labels_country)

In [77]:
data_country.shape

(666, 100)

In [78]:
labels_country.shape

(666,)

In [79]:
#0 for china and 1 for RoW
np.bincount(labels_country)

array([149, 517])

In [81]:
log_reg(data_country,labels_country,2,0.60)

Testing accuracy-
0.7669172932330827


In [146]:
contest = 'Contest1244'
csv = pd.read_csv('../Data/'+str(contest)+'/standings_statistics.csv',sep='\t')#,skiprows=1,header=None,names=['ParticipantID','Rank','Country','ProblemA_id','ProblemA_memory','ProblemA_time','ProblemA_language','ProblemB_id','ProblemB_memory','ProblemB_time','ProblemB_language','ProblemC_id','ProblemC_memory','ProblemC_time','ProblemC_language','ProblemD_id','ProblemD_memory','ProblemD_time','ProblemD_language','ProblemE_id','ProblemE_memory','ProblemE_time','ProblemE_language','ProblemF_id','ProblemF_memory','ProblemF_time','ProblemF_language','ProblemG_id','ProblemG_memory','ProblemG_time','ProblemG_language'])
    

In [92]:
len(np.array(csv.loc[csv['Country']=='China']))

149

In [93]:
len(np.array(csv.loc[(csv['Country']!='China') & (csv['Country']!='-'),['ParticipantID']]))
      

539

### Ground truth = 'Rating

In [108]:
np.array(csv.loc[csv['ParticipantID'].astype(str)=='28934535',['Color']]=='blue')[0][0]

True

In [399]:
x_rating_d2v,labels_rating_d2v = create_dataframe("Contest1244",m,'rating')

In [406]:
x_rating_d2v.shape

(100, 3717)

In [135]:
data_rating = np.array(x_rating)
data_rating = data_rating.T
labels_rating = np.array(labels_rating)

In [136]:
log_reg(data_rating,labels_rating,2,0.6)

Testing accuracy-
0.558843308675185


## Making Vector representation of codes : Tokenization

In [1079]:
cols=['i', '=', 'int', '#', '1', '0', 'n', '.', 'define', '<', 'a', 'x', '<<', '+', 'if', '-', 'll', '()', '++', '>', 'for', 'include', 'b', 'return', 'j', 'long', '>>', '==', '*', '2', 'v', '&', 'c', 'y', 'k', 'ans', 'cin', 'cout', '<=', '%', 'const', 'm', 'T', 'p', 's', 'using', 't', 'std', 'vector', 'else', 'u', 'typedef', 'd', '+=', 'namespace', 'r', 'while', 'N', 'main', 'l', '3', 'f', 'endl', 'void', '/', '::', 'push_back', 'first', 'second', 'size', '<bits/stdc++.h>', 'mod', 'dp', '--', ':', '!=', 'res', '&&', 'pair', 'scanf', 'printf', 'template', 'e', 'cnt', 'w', 'pb', 'tie', 'q', 'ch', 'typename', 'g', 'bool', 'it', 'auto', '>=', 'false', 'sum', 'LL', 'max', 'inline', '!', 'sync_with_stdio', 'double', 'maxn', 'class', 'char', 'long long', 'read', 'MOD', 'begin', 'A', '10', 'break;', 'end', 'rep', '" "', '"%d"', "'\\n'", '...', 'mp', 'pii', 'dfs', '?', 'tmp', 'min', 'getchar', "'0'", 'st', 'sort', 'continue;', 'string', '||', 'cost', '7', '1e9', 'make_pair', 'z', 'solve', 'operator', '"\\n"', '-=', 'all', 'pos', 'ostream', 'os', 'S', 'sz', '<iostream>', 'abs', 'M', '<algorithm>', 'ret', 'ios_base', 'MAXN', 'now', 'arr', 'col', 'ld', 'val', 'true', 'cur', '4', 'find', 'to', 'INF', 'num', "' '", 'vis', 'temp', 'fi', 'vi', 'endif', '5', 'mn', 'se', 'edge', 'out', '1000000007', 'insert', 'gcd', 'F', '<vector>', 'ios', 'B', 'freopen', 'ss', 'R', 'fa', 'par', '<cstdio>', 'P', '%=', 'L', 'id', 'pll', 'root', '1e5', 'inf', '/=', 'pragma', 'mid', "'1'", 'mx', 'C', 'set', 'tot', 'node', "'9'", 'X', 'h', 'adj', '<queue>', 'c2', "'-'", '<map>', 'swap', '<set>', 'c1', 'in', '<cmath>', 'struct', '"%lld\\n"', 'GCC', 'ff', '6', 'signed', 'edges', '100005', '<cstring>', 'stdin', 'G', 'p1', 'FOR', 'vec', '"%lld"', 'return;', 'NULL', 'REP', 'cerr', 'ans1', '1ll', 'o', 'ifdef', 'V', 'K', 'push', 'str', 'erase', '->', '<string>', 'pr', 'U', 'ull', 'color', 'unsigned', '"r"', 'flag', 'ii', '"%d\\n"', 'count', '*=', 'top', 'xx', 'optimize', 'pre', 'get', 'pi', '1e18', 'ans2', 'E', 'puts', 'dist', 'Y', 'nxt', 'fr', 'T1', 'p2', 'len', 'map', 'empty', '[]', 'x1', '1LL', 'lli', 'parent', 'dis', 'add', 'sc', 's1', '<stack>', 'T2', 'print', 'resize', 'stdout', 'used', 'lower_bound', 'head', 'idx', 'PI', 'long double', 'sizeof', 'vll', 'deg', '^', '"-1"', "'('", 're', '", "', 'isdigit', 'debug', 'curr', 'ifndef', 'tree', 'memset', 't1', 'mi', 'a1', 'pair<int,int>', 's2', 'args', '>>=', 'istream', 'dir', 'ind', 'to_string', 'left', '"%d%d"', 'ONLINE_JUDGE', 'putchar', 'eps', '"w"', 'fib', '"-1\\n"', 'power', 'pop', 'right', 'yy', 'upper_bound', 'sum1', 'back', 'ar', 'MAX', 'register', 'names', '<iomanip>', '"%d %d\\n"', 'db', 't2', 'sum2', 'Edge', '""', '<bitset>', 'ok', 'emplace_back', 'pq', '__gcd', 'check', 'start', 'cnt1', 'rt', 'vl', '48', 'x2', 'graph', 'cnt2', 'y1', '0x3f3f3f3f', 'a2', '"%d "', 'tt', 'row', 'cmp', '"input.txt"', 'Q', 'md', '{}', 'int32_t', 'init', '__gnu_pbds', 'D', '2005', 'Args', 'from', 'write', 'base', 'last', 'Arg1', 'pp', 'acos', '<unordered_map>', 'ed', '<ext/pb_ds/assoc_container.hpp>', 'next', 'nullptr', '1e6', 'prime', 'front', 'long long int', 'forn', '100010']


In [1080]:
contest_list = ['Contest1236','Contest1243','Contest1244','Contest1245','Contest1248']
token_data_list=[]
for i in contest_list:
    token_data_list.append(pd.read_csv('../Data/'+str(i)+'/tokens_stats.txt',sep=' ',header=None,skiprows=1,names=cols))
token_data = pd.concat(token_data_list)
token_data = (token_data - token_data.mean())/token_data.std()
csv_list=[]
for i in contest_list:
    csv_list.append(pd.read_csv('../Data/'+str(i)+'/standings_statistics.csv',sep='\t'))
csv = pd.concat(csv_list,sort=True)
     
China_participants = np.array(csv.loc[(csv['Country']=='China'),['ParticipantID']])
China_participants = [str(i[0]) for i in China_participants]
RoW_participants = np.array(csv.loc[(csv['Country']!='China') & (csv['Country']!='-'),['ParticipantID']])
RoW_participants =  [str(i[0]) for i in RoW_participants]

code_files = list(token_data.index)
pid = [ i.split('_')[0] for i in code_files]
token_data_2 = token_data.assign(pid=pid) 

In [1081]:
token_data.shape

(17866, 400)

#### GRound truth = Country

In [1082]:
def check_region(x):
    if x in China_participants:
        return 0
    elif x in RoW_participants:
        return 1
    else:
        return 2

    
    

In [1083]:
token_data_2['region'] = token_data_2['pid'].apply(check_region)

In [1084]:
token_data_2['region'].value_counts()
token_data_2 = token_data_2[token_data_2.region != 2]


In [1085]:
token_data_2.drop('pid',axis=1,inplace=True)

In [1086]:
stopwords=['ans', 'm', 'T', 'p', 's', 't',  'u',  'd',  'r',  'N',  'l', '3', 'f', 'dp', 'res',  'e', 'cnt', 'w', 'q', 'ch','g', 'it',   'A', '10', 'tmp', 'st', '7', '1e9',  'z',  'all', 'S', 'sz',  'M', 'MAXN', 'now', 'arr', 'col',  'val', 'cur', '4', 'num',  'vis', 'temp', '5', 'mn', 'se', 'F','B', 'ss', 'R', 'fa', 'par', 'P', 'L', 'id', 'root', '1e5', 'inf', 'mid', "'1'", 'mx', 'C', 'tot', 'node', "'9'", 'X', 'h', 'adj', 'c2', 'c1',  '6',  'edges', '100005', 'G', 'p1', 'vec','ans1','o', 'V', 'K', 'pr', 'color','flag',  'count','top', 'xx',  'pre', 'ans2', 'E', 'dist', 'Y', 'T1', 'p2', 'len', 'x1','parent', 'sc', 's1',  'T2',   'head', 'idx', 'PI', 'deg', '^', '"-1"', "'('", '", "', 'curr', 'tree',  't1', 'mi', 'a1','s2', 'dir', 'ind',  'left',  'eps', '"w"', 'fib', '"-1\\n"',  'right', 'yy', 'sum1', 'ar' , 'names', 'db', 't2', 'sum2', 'Edge', '""', 'ok', 'check', 'start', 'cnt1', 'rt', 'vl', '48', 'x2', 'graph', 'cnt2', 'y1','0x3f3f3f3f','a2',  'tt', 'row', 'cmp', '"input.txt"', 'Q', 'md',  'D', '2005', 'Args', '1e6',  'forn', '100010','k','c','x']

In [1087]:
for i in stopwords:
    token_data_2.drop(i,axis=1,inplace=True)

In [1088]:
token_data_2.shape
col_red = token_data_2.columns

In [1090]:
token_data_2.shape

(11980, 243)

In [1091]:
col_red =col_red[:242]

In [1093]:
token_data_2.head()

,i,=,int,#,1,0,n,.,define,<,...,acos,<unordered_map>,ed,<ext/pb_ds/assoc_container.hpp>,next,nullptr,prime,front,long long int,region
29019536_62792024,-0.874050,-0.333972,2.738474,-0.458847,-0.436629,-0.331151,-0.314934,3.478957,1.753606,-0.084841,...,-0.145581,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,1
29019536_62796810,-0.234478,0.260540,2.738474,0.085421,0.201396,-0.197421,0.521314,3.478957,1.753606,0.126210,...,-0.145581,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,1
29019599_62783989,-0.874050,-0.705543,0.066734,-0.567701,-0.755641,-0.598612,-0.983932,0.417174,-0.243637,-0.506943,...,-0.145581,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,1
29019599_62788377,-0.874050,-0.482601,0.066734,-0.894261,-0.436629,-0.598612,-0.482183,0.417174,0.108818,-0.506943,...,-0.145581,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,1
29019599_62798239,-0.170521,-0.333972,0.066734,-0.349993,-0.330291,0.070039,0.688563,0.417174,0.108818,0.020685,...,-0.145581,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,1


In [1094]:
x = token_data_2.iloc[:,:243]

In [1096]:
y = token_data_2.iloc[:,242:]
y = np.array(y)
y = [y[i][0] for i in range(len(y))]
y = np.array(y)


In [1097]:
np.bincount(y)

array([2928, 9052])

In [1098]:
x_train,y_train,x_test,y_test,class_weight = train_test_split(x,y,2,0.6)
log_reg(x_train,y_train,x_test,y_test,2,class_weight)

Training accuracy-
1.0
Testing accuracy-
0.9993739565943238
Confusion matrix -
[[1547    2]
 [   1 3242]]


LogisticRegression(C=1.0, class_weight={0: 1.0, 1: 1.0}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [1099]:
rf_country = rw(x_train,y_train,x_test,y_test,class_weight)

Training accuracy-
1.0
Testing accuracy-
1.0
Confusion matrix -
[[1549    0]
 [   0 3243]]


In [1100]:
coeffecient_importance = rf_country.feature_importances_

In [1101]:
f_imp_score={}
for i in range(len(col_red)):
    f_imp_score[col_red[i]]=coeffecient_importance[i]

In [1102]:
import operator
sorted_f_imp_score = sorted(f_imp_score.items(),key=operator.itemgetter(1),reverse=True)

In [1103]:
sorted_f_imp_score

[('long', 0.05532138678700934),
 ('...', 0.019873680333256073),
 ('.', 0.016058044259853993),
 ('cost', 0.015920175087399426),
 ('mod', 0.014685800660481642),
 ('template', 0.01455949957210428),
 ('<queue>', 0.013395722056886704),
 ('y', 0.011647758008931279),
 ('a', 0.010567760449858575),
 ('GCC', 0.010547218964351596),
 ('int', 0.009444801705375152),
 ('::', 0.00937568692228093),
 ('<', 0.008562068455663348),
 ('-=', 0.007989833796084278),
 ('false', 0.007655572139876926),
 ('solve', 0.007595442891479202),
 ('min', 0.007320865753412682),
 ('read', 0.006774550484154715),
 ('tie', 0.006122253678526306),
 ('1', 0.0059945303552716486),
 ('0', 0.005989623227423299),
 ('else', 0.005977605898838623),
 ('=', 0.005535702504097285),
 ('<=', 0.005378793828063295),
 ('const', 0.00531301364368242),
 ('"r"', 0.005305471027365008),
 ('#', 0.005191129046647163),
 ('endl', 0.005181092759784093),
 ('i', 0.004938741464583401),
 ('()', 0.004883113973178178),
 ('include', 0.004880223525302169),
 ('j', 0.

### Rating

In [1105]:
def check_rating(x):
    if x in blue_participants:
        return 0
    elif x in violet_participants:
        return 1
#     elif x in violet_participants:
#         return 2
    else :
        return 3

In [1106]:
contest_list = ['Contest1236','Contest1243','Contest1244','Contest1245','Contest1248']
token_data_list=[]
for i in contest_list:
    token_data_list.append(pd.read_csv('../Data/'+str(i)+'/tokens_stats.txt',sep=' ',header=None,skiprows=1,names=cols))
token_data = pd.concat(token_data_list)
token_data = (token_data - token_data.mean())/token_data.std()

csv_list=[]
for i in contest_list:
    csv_list.append(pd.read_csv('../Data/'+str(i)+'/standings_statistics.csv',sep='\t'))
csv = pd.concat(csv_list,sort=True)
     
blue_participants = np.array(csv.loc[(csv['Color']=='blue'),['ParticipantID']])
blue_participants = [str(i[0]) for i in blue_participants]
cyan_participants = np.array(csv.loc[(csv['Color']=='cyan'),['ParticipantID']]) #& (csv['Country']!='-'),['ParticipantID']])
cyan_participants =  [str(i[0]) for i in cyan_participants]
violet_participants = np.array(csv.loc[(csv['Color']=='violet'),['ParticipantID']]) #& (csv['Country']!='-'),['ParticipantID']])
violet_participants =  [str(i[0]) for i in violet_participants]

code_files = list(token_data.index)
pid = [ i.split('_')[0] for i in code_files]
token_data_2 = token_data.assign(pid=pid) 

In [1107]:
token_data_2['rating'] = token_data_2['pid'].apply(check_rating)
stopwords=['ans', 'm', 'T', 'p', 's', 't',  'u',  'd',  'r',  'N',  'l', '3', 'f', 'dp', 'res',  'e', 'cnt', 'w', 'q', 'ch','g', 'it',   'A', '10', 'tmp', 'st', '7', '1e9',  'z',  'all', 'S', 'sz',  'M', 'MAXN', 'now', 'arr', 'col',  'val', 'cur', '4', 'num',  'vis', 'temp', '5', 'mn', 'se', 'F','B', 'ss', 'R', 'fa', 'par', 'P', 'L', 'id', 'root', '1e5', 'inf', 'mid', "'1'", 'mx', 'C', 'tot', 'node', "'9'", 'X', 'h', 'adj', 'c2', 'c1',  '6',  'edges', '100005', 'G', 'p1', 'vec','ans1','o', 'V', 'K', 'pr', 'color','flag',  'count','top', 'xx',  'pre', 'ans2', 'E', 'dist', 'Y', 'T1', 'p2', 'len', 'x1','parent', 'sc', 's1',  'T2',   'head', 'idx', 'PI', 'deg', '^', '"-1"', "'('", '", "', 'curr', 'tree',  't1', 'mi', 'a1','s2', 'dir', 'ind',  'left',  'eps', '"w"', 'fib', '"-1\\n"',  'right', 'yy', 'sum1', 'ar' , 'names', 'db', 't2', 'sum2', 'Edge', '""', 'ok', 'check', 'start', 'cnt1', 'rt', 'vl', '48', 'x2', 'graph', 'cnt2', 'y1','0x3f3f3f3f','a2',  'tt', 'row', 'cmp', '"input.txt"', 'Q', 'md',  'D', '2005', 'Args', '1e6',  'forn', '100010','k','c','x']
for i in stopwords:
    token_data_2.drop(i,axis=1,inplace=True)
col_red = token_data_2.columns

In [1108]:
token_data_2.head()

,i,=,int,#,1,0,n,.,define,<,...,<unordered_map>,ed,<ext/pb_ds/assoc_container.hpp>,next,nullptr,prime,front,long long int,pid,rating
29019536_62792024,-0.874050,-0.333972,2.738474,-0.458847,-0.436629,-0.331151,-0.314934,3.478957,1.753606,-0.084841,...,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,29019536,0
29019536_62796810,-0.234478,0.260540,2.738474,0.085421,0.201396,-0.197421,0.521314,3.478957,1.753606,0.126210,...,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,29019536,0
29019599_62783989,-0.874050,-0.705543,0.066734,-0.567701,-0.755641,-0.598612,-0.983932,0.417174,-0.243637,-0.506943,...,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,29019599,1
29019599_62788377,-0.874050,-0.482601,0.066734,-0.894261,-0.436629,-0.598612,-0.482183,0.417174,0.108818,-0.506943,...,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,29019599,1
29019599_62798239,-0.170521,-0.333972,0.066734,-0.349993,-0.330291,0.070039,0.688563,0.417174,0.108818,0.020685,...,-0.077121,-0.167893,-0.191163,-0.192961,-0.076856,-0.113487,-0.106641,-0.128807,29019599,1


In [1109]:
token_data_2['rating'].value_counts()

0    10819
3     4060
1     2987
Name: rating, dtype: int64

In [1110]:
token_data_2['rating'].value_counts()
token_data_2 = token_data_2[token_data_2.rating != 3]

In [1111]:
token_data_2.drop('pid',axis=1,inplace=True)

In [1114]:
token_data_2.shape

(13806, 243)

In [1115]:
x = token_data_2.iloc[:,:243]

In [1116]:
y = token_data_2.iloc[:,242:]
y = np.array(y)
y = [y[i][0] for i in range(len(y))]
y=np.array(y)

In [1117]:
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [1118]:
x_train_rating,y_train_rating,x_test_rating,y_test_rating,class_weight_rating = train_test_split(x,y,2,0.5)


In [1119]:
m_lr = log_reg(x_train_rating,y_train_rating,x_test_rating,y_test_rating,2,class_weight_rating)

Training accuracy-
1.0
Testing accuracy-
1.0
Confusion matrix -
[[5251    0]
 [   0 1652]]


#### RandomForest

In [1120]:
from sklearn.ensemble import RandomForestClassifier

In [1121]:
rf_rating = rw(x_train_rating,y_train_rating,x_test_rating,y_test_rating,class_weight_rating)

Training accuracy-
1.0
Testing accuracy-
0.9955091988990294
Confusion matrix -
[[5251    0]
 [  31 1621]]


In [1126]:
coeffecient_importance_rating = rf_rating.feature_importances_
col_red = col_red[:242]

In [1127]:
f_imp_score_rating={}
for i in range(len(col_red)):
    f_imp_score_rating[col_red[i]]=coeffecient_importance_rating[i]

In [1128]:
import operator
sorted_f_imp_score_rating = sorted(f_imp_score.items(),key=operator.itemgetter(1),reverse=True)
sorted_f_imp_score_rating

[('long', 0.05532138678700934),
 ('...', 0.019873680333256073),
 ('.', 0.016058044259853993),
 ('cost', 0.015920175087399426),
 ('mod', 0.014685800660481642),
 ('template', 0.01455949957210428),
 ('<queue>', 0.013395722056886704),
 ('y', 0.011647758008931279),
 ('a', 0.010567760449858575),
 ('GCC', 0.010547218964351596),
 ('int', 0.009444801705375152),
 ('::', 0.00937568692228093),
 ('<', 0.008562068455663348),
 ('-=', 0.007989833796084278),
 ('false', 0.007655572139876926),
 ('solve', 0.007595442891479202),
 ('min', 0.007320865753412682),
 ('read', 0.006774550484154715),
 ('tie', 0.006122253678526306),
 ('1', 0.0059945303552716486),
 ('0', 0.005989623227423299),
 ('else', 0.005977605898838623),
 ('=', 0.005535702504097285),
 ('<=', 0.005378793828063295),
 ('const', 0.00531301364368242),
 ('"r"', 0.005305471027365008),
 ('#', 0.005191129046647163),
 ('endl', 0.005181092759784093),
 ('i', 0.004938741464583401),
 ('()', 0.004883113973178178),
 ('include', 0.004880223525302169),
 ('j', 0.

### Doc2vec + token

In [783]:
#ground truth = rating

In [784]:
contest_list = ['Contest1236','Contest1243','Contest1244','Contest1245','Contest1248']
token_data_list=[]
for i in contest_list:
    token_data_list.append(pd.read_csv('../Data/'+str(i)+'/tokens_stats.txt',sep=' ',header=None,skiprows=1,names=cols))
token_data = pd.concat(token_data_list)
token_data = (token_data - token_data.mean())/token_data.std()

csv_list=[]
for i in contest_list:
    csv_list.append(pd.read_csv('../Data/'+str(i)+'/standings_statistics.csv',sep='\t'))
csv = pd.concat(csv_list,sort=True)
     
blue_participants = np.array(csv.loc[(csv['Color']=='blue'),['ParticipantID']])
blue_participants = [str(i[0]) for i in blue_participants]
cyan_participants = np.array(csv.loc[(csv['Color']=='cyan'),['ParticipantID']]) #& (csv['Country']!='-'),['ParticipantID']])
cyan_participants =  [str(i[0]) for i in cyan_participants]
violet_participants = np.array(csv.loc[(csv['Color']=='violet'),['ParticipantID']]) #& (csv['Country']!='-'),['ParticipantID']])
violet_participants =  [str(i[0]) for i in violet_participants]

code_files = list(token_data.index)
pid = [ i.split('_')[0] for i in code_files]
token_data_2 = token_data.assign(pid=pid) 

In [785]:
token_data_2.head()

,i,=,int,#,1,0,n,.,define,<,...,<ext/pb_ds/assoc_container.hpp>,next,nullptr,1e6,prime,front,long long int,forn,100010,pid
29019536_62792024,-0.874050,-0.333972,2.738474,-0.458847,-0.436629,-0.331151,-0.314934,3.478957,1.753606,-0.084841,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019536
29019536_62796810,-0.234478,0.260540,2.738474,0.085421,0.201396,-0.197421,0.521314,3.478957,1.753606,0.126210,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019536
29019599_62783989,-0.874050,-0.705543,0.066734,-0.567701,-0.755641,-0.598612,-0.983932,0.417174,-0.243637,-0.506943,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599
29019599_62788377,-0.874050,-0.482601,0.066734,-0.894261,-0.436629,-0.598612,-0.482183,0.417174,0.108818,-0.506943,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599
29019599_62798239,-0.170521,-0.333972,0.066734,-0.349993,-0.330291,0.070039,0.688563,0.417174,0.108818,0.020685,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599


In [832]:
contest = 'Contest1244'
test = pd.read_csv('../Data/'+str(contest)+'/doc2vec_embedding_model_100.txt',sep='\t',header=None)

In [1018]:

def d2v(e_size):
    d2v_data_list=[]
    for i in contest_list:
        #x=pd.read_csv('../Data/'+str(i)+'/doc2vec_embedding_model_'+str(e_size)+'.txt',sep='\t',header=None)
        
        d2v_data_list.append(pd.read_csv('../Data/'+str(i)+'/doc2vec_embedding_model_'+str(e_size)+'.txt',sep='\t',header=None))
    d2v_vec = pd.concat(d2v_data_list)#,sort=True)
    return d2v_vec


In [1019]:
d2v_data = d2v(300)

In [1020]:
d2v_data.shape

(17871, 301)

In [1021]:
d2v_data.columns

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            291, 292, 293, 294, 295, 296, 297, 298, 299, 300],
           dtype='int64', length=301)

In [1033]:
d2v_data.head()

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,300,xml
0,29019536_62784831.xml,-0.093141,-0.056699,0.188417,-0.144539,-0.196943,0.012109,-0.108803,0.110863,0.021378,...,-0.058394,-0.064447,0.079190,0.044644,-0.080906,0.025222,-0.011280,0.076119,-0.039206,29019536_62784831
1,29019536_62792024.xml,-0.075383,0.001009,0.198052,-0.201404,-0.199120,-0.048383,-0.114775,0.079104,0.044544,...,-0.081014,0.034650,0.078785,0.087769,-0.100715,0.026909,0.026203,0.084212,-0.006191,29019536_62792024
2,29019536_62796810.xml,-0.138566,-0.062372,0.191516,-0.231664,-0.312279,-0.100677,-0.262337,0.078280,0.127548,...,-0.046765,0.170347,0.213926,-0.065348,-0.057487,-0.007682,0.049086,-0.099233,0.080386,29019536_62796810
3,29019599_62783989.xml,0.026718,-0.322861,-0.027545,-0.025447,0.106338,0.189130,-0.182498,-0.053114,-0.038645,...,0.190087,-0.199105,0.063345,0.159150,0.134291,0.086986,-0.099186,0.165730,0.086032,29019599_62783989
4,29019599_62788377.xml,0.030715,-0.321491,-0.039830,0.010788,0.155476,0.169093,-0.157595,0.000212,0.000922,...,0.160540,-0.214712,0.052052,0.183481,0.058735,0.047098,-0.071226,0.131127,0.064378,29019599_62788377


In [1035]:
code_files = d2v_data[0].to_list()
pid = [ i.split('_')[0] for i in code_files]
d2v_data = d2v_data.assign(pid=pid)

In [1037]:
d2v_data['rating'] = d2v_data['pid'].apply(check_rating)

In [1038]:
d2v_data.head()

,0,1,2,3,4,5,6,7,8,9,...,294,295,296,297,298,299,300,xml,pid,rating
0,29019536_62784831.xml,-0.093141,-0.056699,0.188417,-0.144539,-0.196943,0.012109,-0.108803,0.110863,0.021378,...,0.079190,0.044644,-0.080906,0.025222,-0.011280,0.076119,-0.039206,29019536_62784831,29019536,0
1,29019536_62792024.xml,-0.075383,0.001009,0.198052,-0.201404,-0.199120,-0.048383,-0.114775,0.079104,0.044544,...,0.078785,0.087769,-0.100715,0.026909,0.026203,0.084212,-0.006191,29019536_62792024,29019536,0
2,29019536_62796810.xml,-0.138566,-0.062372,0.191516,-0.231664,-0.312279,-0.100677,-0.262337,0.078280,0.127548,...,0.213926,-0.065348,-0.057487,-0.007682,0.049086,-0.099233,0.080386,29019536_62796810,29019536,0
3,29019599_62783989.xml,0.026718,-0.322861,-0.027545,-0.025447,0.106338,0.189130,-0.182498,-0.053114,-0.038645,...,0.063345,0.159150,0.134291,0.086986,-0.099186,0.165730,0.086032,29019599_62783989,29019599,1
4,29019599_62788377.xml,0.030715,-0.321491,-0.039830,0.010788,0.155476,0.169093,-0.157595,0.000212,0.000922,...,0.052052,0.183481,0.058735,0.047098,-0.071226,0.131127,0.064378,29019599_62788377,29019599,1


In [1005]:
token_data_2['xml']=token_data_2.index

In [1006]:
token_data_2.head()

,i,=,int,#,1,0,n,.,define,<,...,next,nullptr,1e6,prime,front,long long int,forn,100010,pid,xml
29019536_62792024,-0.874050,-0.333972,2.738474,-0.458847,-0.436629,-0.331151,-0.314934,3.478957,1.753606,-0.084841,...,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019536,29019536_62792024
29019536_62796810,-0.234478,0.260540,2.738474,0.085421,0.201396,-0.197421,0.521314,3.478957,1.753606,0.126210,...,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019536,29019536_62796810
29019599_62783989,-0.874050,-0.705543,0.066734,-0.567701,-0.755641,-0.598612,-0.983932,0.417174,-0.243637,-0.506943,...,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599,29019599_62783989
29019599_62788377,-0.874050,-0.482601,0.066734,-0.894261,-0.436629,-0.598612,-0.482183,0.417174,0.108818,-0.506943,...,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599,29019599_62788377
29019599_62798239,-0.170521,-0.333972,0.066734,-0.349993,-0.330291,0.070039,0.688563,0.417174,0.108818,0.020685,...,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599,29019599_62798239


In [986]:
joined_vec = pd.merge(x,token_data_2,on='xml')

In [989]:
joined_vec.head()

,1,2,3,4,5,6,7,8,9,10,...,<ext/pb_ds/assoc_container.hpp>,next,nullptr,1e6,prime,front,long long int,forn,100010,pid
0,-0.075383,0.001009,0.198052,-0.201404,-0.199120,-0.048383,-0.114775,0.079104,0.044544,-0.057835,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019536
1,-0.138566,-0.062372,0.191516,-0.231664,-0.312279,-0.100677,-0.262337,0.078280,0.127548,0.101957,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019536
2,0.026718,-0.322861,-0.027545,-0.025447,0.106338,0.189130,-0.182498,-0.053114,-0.038645,-0.108236,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599
3,0.030715,-0.321491,-0.039830,0.010788,0.155476,0.169093,-0.157595,0.000212,0.000922,-0.104182,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599
4,-0.036539,-0.284387,-0.125051,-0.062587,0.092303,0.097412,-0.232940,0.098014,0.094612,0.007199,...,-0.191163,-0.192961,-0.076856,-0.10853,-0.113487,-0.106641,-0.128807,-0.203576,-0.120868,29019599


In [990]:
joined_vec['rating'] = joined_vec['pid'].apply(check_rating)

In [991]:
joined_vec['rating'].value_counts()

0    10819
3     4060
1     2987
Name: rating, dtype: int64

In [992]:
joined_vec = joined_vec[joined_vec.rating != 3]
joined_vec.drop('pid',axis=1,inplace=True)




In [993]:
joined_vec.drop('xml',axis=1,inplace=True)

In [994]:
joined_vec.shape

(13806, 701)

In [995]:
x = joined_vec.iloc[:,:700]
y = joined_vec.iloc[:,700:]
y = np.array(y)
y = [y[i][0] for i in range(len(y))]
y=np.array(y)

In [996]:
x.shape

(13806, 700)

In [997]:
y.shape

(13806,)

In [998]:
x_train_rating,y_train_rating,x_test_rating,y_test_rating,class_weight_rating = train_test_split(x,y,2,0.5)

In [999]:
m_lr = log_reg(x_train_rating,y_train_rating,x_test_rating,y_test_rating,2,class_weight_rating)

Training accuracy-
0.8561495002172969
Testing accuracy-
0.6905693176879617
Confusion matrix -
[[4444  805]
 [1331  323]]


In [1039]:
d2v_data = d2v_data[d2v_data.rating != 3]
d2v_data.drop('pid',axis=1,inplace=True)
d2v_data.drop(0,axis=1,inplace=True)
d2v_data.drop('xml',axis=1,inplace=True)

/home/ashish-j/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [1041]:
x = d2v_data.iloc[:,:300]
y = d2v_data.iloc[:,300:]
y = np.array(y)
y = [y[i][0] for i in range(len(y))]
y=np.array(y)

In [1042]:
x_train_rating,y_train_rating,x_test_rating,y_test_rating,class_weight_rating = train_test_split(x,y,2,0.5)

In [1043]:
m_lr = log_reg(x_train_rating,y_train_rating,x_test_rating,y_test_rating,2,class_weight_rating)

Training accuracy-
0.8078204199855178
Testing accuracy-
0.7345401882693701
Confusion matrix -
[[5027  223]
 [1610   45]]
